In [1]:
import nltk
from nltk.tokenize import sent_tokenize
import spacy
from nltk.stem import WordNetLemmatizer

In [2]:
nlp = spacy.load('en_core_web_sm')
lemmatizer = WordNetLemmatizer()
textfile = "data/set4/a7.txt"
#put entire text file into a list of sentences
text = []
with open(textfile, "r") as f:
	for line in f:
		line = line.split('. ')
		if len(line) != 0:
			temp = line[0].strip('\n')
			if len(temp) != 0:
				text.append(temp)


In [4]:
print(nlp("hi y name"))

hi y name


In [11]:
print(pos_tag_lst("hello my name is Laura"))

{0: [('h', 'X', 'LS', 'ROOT', False)], 1: [('e', 'VERB', 'VB', 'ROOT', False)], 2: [('l', 'NOUN', 'NN', 'ROOT', False)], 3: [('l', 'NOUN', 'NN', 'ROOT', False)], 4: [('o', 'INTJ', 'UH', 'ROOT', False)], 5: [(' ', 'SPACE', '_SP', 'ROOT', False)], 6: [('m', 'VERB', 'VBP', 'ROOT', False)], 7: [('y', 'PROPN', 'NNP', 'ROOT', False)], 8: [(' ', 'SPACE', '_SP', 'ROOT', False)], 9: [('n', 'ADV', 'RB', 'ROOT', False)], 10: [('a', 'DET', 'DT', 'ROOT', True)], 11: [('m', 'VERB', 'VBP', 'ROOT', False)], 12: [('e', 'VERB', 'VB', 'ROOT', False)], 13: [(' ', 'SPACE', '_SP', 'ROOT', False)], 14: [('i', 'PRON', 'PRP', 'ROOT', True)], 15: [('s', 'VERB', 'VBZ', 'ROOT', False)], 16: [(' ', 'SPACE', '_SP', 'ROOT', False)], 17: [('L', 'NOUN', 'NN', 'ROOT', False)], 18: [('a', 'DET', 'DT', 'ROOT', True)], 19: [('u', 'NOUN', 'NN', 'ROOT', False)], 20: [('r', 'NOUN', 'NN', 'ROOT', False)], 21: [('a', 'DET', 'DT', 'ROOT', True)]}


In [10]:
def pos_tag_lst(text):
	#list of sentences
	POS_tag_dict = dict()
	for i,line in enumerate(text):
		tags = []
		doc = nlp(str(line))
        print(doc)
		for token in doc:
#             print(doc)
#             print(token)
			tags.append((token.text, token.pos_, token.tag_, token.dep_, token.is_stop))
		if len(tags) != 0:
			POS_tag_dict[i] = tags
	return POS_tag_dict

def pos_tag_sentence(sentence):
	#list of sentences
	POS_tag_dict = dict()
	text = sentence.split()
	for line in text:
		tags = []
		doc = nlp(str(line))
		for token in doc:
			tags.append((token.pos_, token.tag_, token.dep_, token.is_stop, ))
		if len(tags) != 0:
			POS_tag_dict[token.text] = tags[0]
	return POS_tag_dict

#Token dict
def dependency_dict(doc):
	out = dict()
	root = ''
	for token in doc:
		out[token.text] = (token.dep_, token.head.text, token.head.pos_,[child for child in token.children])
		if token.dep_ == "ROOT":
			root = token.text
	return out, root

# NER tagging
def ner_tag(text):
	NER_tag_dict = dict()
	for i,line in enumerate(text):
		tags = []
		doc = nlp(str(line))

		for ent in doc.ents:
			# print(ent.text +'-' + ent.label_ + '\n')
			tags.append(ent.text +'-' + ent.label_)
		if len(tags) != 0:
			NER_tag_dict[i] = tags
	return NER_tag_dict

def ner_tag_sentence(sentence):
	doc = nlp(str(sentence))
	NER_tag_dict = dict()
	tags = []
	for ent in doc.ents:
		# print(ent.text +'-' + ent.label_ + '\n')
		NER_tag_dict[ent.text] = ent.label_
	return NER_tag_dict


#check tense of verb
def check_tense(root, pos_dict):
	tag = pos_dict[root][1]
	if tag == "VB":
		return "do"
	elif tag == "VBD":
		return "did"
	elif tag == "VBG":
		return "doing"
	elif tag == "VBN":
		return "done"
	elif tag == "VBP":
		return "do"
	elif tag == "VBZ":
		return "does"
	else:
		return None

# return dictionary of lemmas in sentence
def getTokenLemma(nlp_doc):
	lemmas = {}
	for token in nlp_doc:
		lemmas[str(token)] = token.lemma_
	return lemmas


TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-10-855aa09a104e>, line 7)

In [19]:
def whereQ(sentence, dep_dict, root):
    output = ''
    verbs = ['was', 'is', 'were']
    foundVerb = False
    foundVerbInd = 0
    # find tense 
    pos_tags = pos_tag_sentence(sentence)
    seenWhere = False
    foundWhereInd = 0
    whereInd = 0
    tense = None
    for word in sentence.split():
        if word == 'where':
            seenWhere = True
            foundWhereInd = whereInd
            verb = dep_dict[word][1]
            if pos_tags[verb][0] == 'VERB':
                tense = check_tense(verb, pos_tags)
            elif verb in verbs: # if tense is was, is, were
                tense = verb
                foundVerb = True
                foundVerbInd = sentence.split().index(verb)
            break
        whereInd += 1
    if tense == None: # not able to create question from sentence
        return "No question"
    output += f'Where {tense} '
    ind = foundWhereInd
    if foundVerb:
        ind = foundVerbInd
    for word in sentence.split()[ind+1:]:
        output += word + " "    
    return output[:-2] + "?"    

In [7]:
sentence = "This leads the trio to an underground passage to the Shrieking Shack, where they discover that the dog is actually Sirius, who is an Animagus."
dep_dict1, root1 = dependency_dict(nlp(sentence))
print(dep_dict1, root1)

{'This': ('nsubj', 'leads', 'VERB', []), 'leads': ('ROOT', 'leads', 'VERB', [This, trio, to, .]), 'the': ('det', 'dog', 'NOUN', []), 'trio': ('dobj', 'leads', 'VERB', [the]), 'to': ('prep', 'passage', 'NOUN', [Shack]), 'an': ('det', 'Animagus', 'PROPN', []), 'underground': ('amod', 'passage', 'NOUN', []), 'passage': ('pobj', 'to', 'ADP', [an, underground, to]), 'Shrieking': ('compound', 'Shack', 'PROPN', []), 'Shack': ('pobj', 'to', 'ADP', [the, Shrieking, ,, discover]), ',': ('punct', 'Sirius', 'PROPN', []), 'where': ('advmod', 'discover', 'VERB', []), 'they': ('nsubj', 'discover', 'VERB', []), 'discover': ('relcl', 'Shack', 'PROPN', [where, they, is]), 'that': ('mark', 'is', 'AUX', []), 'dog': ('nsubj', 'is', 'AUX', [the]), 'is': ('relcl', 'Sirius', 'PROPN', [who, Animagus]), 'actually': ('advmod', 'is', 'AUX', []), 'Sirius': ('attr', 'is', 'AUX', [,, is]), 'who': ('nsubj', 'is', 'VERB', []), 'Animagus': ('attr', 'is', 'VERB', [an]), '.': ('punct', 'leads', 'VERB', [])} leads


In [10]:
sentence2 = "He was in the underground, where there were lots of tea."
dep_dict2, root2 = dependency_dict(nlp(sentence2))
print(dep_dict2, root2)

{'He': ('nsubj', 'was', 'AUX', []), 'was': ('ROOT', 'was', 'AUX', [He, in, .]), 'in': ('prep', 'was', 'AUX', [underground]), 'the': ('det', 'underground', 'NOUN', []), 'underground': ('pobj', 'in', 'ADP', [the, ,, were]), ',': ('punct', 'underground', 'NOUN', []), 'where': ('advmod', 'were', 'VERB', []), 'there': ('expl', 'were', 'VERB', []), 'were': ('relcl', 'underground', 'NOUN', [where, there, lots]), 'lots': ('attr', 'were', 'VERB', [of]), 'of': ('prep', 'lots', 'NOUN', [tea]), 'tea': ('pobj', 'of', 'ADP', []), '.': ('punct', 'was', 'AUX', [])} was


In [11]:
pos_tag_sentence(sentence2)

{'He': ('PRON', 'PRP', 'ROOT', True),
 'was': ('AUX', 'VBD', 'ROOT', True),
 'in': ('ADP', 'IN', 'ROOT', True),
 'the': ('DET', 'DT', 'ROOT', True),
 ',': ('ADV', 'RB', 'ROOT', False),
 'where': ('ADV', 'WRB', 'ROOT', True),
 'there': ('ADV', 'RB', 'ROOT', True),
 'were': ('AUX', 'VBD', 'ROOT', True),
 'lots': ('NOUN', 'NNS', 'ROOT', False),
 'of': ('ADP', 'IN', 'ROOT', True),
 '.': ('NOUN', 'NN', 'ROOT', False)}

In [9]:
pos_tag_sentence(sentence)

{'This': ('DET', 'DT', 'ROOT', True),
 'leads': ('VERB', 'VBZ', 'ROOT', False),
 'the': ('DET', 'DT', 'ROOT', True),
 'trio': ('NOUN', 'NN', 'ROOT', False),
 'to': ('PART', 'TO', 'ROOT', True),
 'an': ('DET', 'DT', 'ROOT', True),
 'underground': ('ADV', 'RB', 'ROOT', False),
 'passage': ('NOUN', 'NN', 'ROOT', False),
 'Shrieking': ('VERB', 'VBG', 'ROOT', False),
 ',': ('PROPN', 'NNP', 'ROOT', False),
 'where': ('ADV', 'WRB', 'ROOT', True),
 'they': ('PRON', 'PRP', 'ROOT', True),
 'discover': ('VERB', 'VB', 'ROOT', False),
 'that': ('DET', 'DT', 'ROOT', True),
 'dog': ('NOUN', 'NN', 'ROOT', False),
 'is': ('AUX', 'VBZ', 'ROOT', True),
 'actually': ('ADV', 'RB', 'ROOT', False),
 'who': ('PRON', 'WP', 'ROOT', True),
 '.': ('PROPN', 'NNP', 'ROOT', False)}

In [15]:
print(whereQ(sentence, dep_dict1, root1))


Where do they discover that the dog is actually Sirius, who is an Animagus?


In [20]:
print(whereQ(sentence2, dep_dict2, root2))

Where were lots of tea?
